In [23]:
from llama_index.core import Document
import requests
from datetime import datetime, timedelta
import time
import feedparser

def get_daily_arxiv_papers():
    max_results = 25
    categories = ["cs.AI", "cs.CV", "cs.IR", "cs.LG", "cs.CL"]
    base_url = "http://export.arxiv.org/api/query?"
    all_categories = [f"cat:{category}" for category in categories]
    search_query = "+OR+".join(all_categories)

    paper_list = []
    start = 0
    today = datetime.utcnow().date()
    new_papers_found = True
    wait_time = 3

    while new_papers_found:
        query = f"search_query={search_query}&start={start}&max_results={max_results}&sortBy=submittedDate&sortOrder=descending"
        response = requests.get(base_url + query)
        feed = feedparser.parse(response.content)

        for r in feed.entries:
            paper_date = datetime.strptime(r["published"][:10], "%Y-%m-%d").date()
            if paper_date >= today - timedelta(1):
                new_papers_found = True
                paper_list.append(
                    Document(
                        text=f"""
Title: {r['title']}
{r['summary']}
                """,
                        metadata={
                            "paper_id": r["id"].split("/")[-1],
                            "paper_link": r["id"],
                            "title": r["title"],
                            "date": r["published"][:10],
                        },
                    )
                )
            else:
                new_papers_found = False
                break

        start += max_results
        time.sleep(wait_time)

    return paper_list

In [24]:
paper_list = get_daily_arxiv_papers()

In [26]:
from llama_index.core.schema import MetadataMode
daily_paper_content = "\n===============\n".join([paper.get_content(MetadataMode.LLM) for paper in paper_list])

In [28]:
from llama_index.llms.gemini import Gemini
import os
from dotenv import load_dotenv

load_dotenv(override=True)

gemini = Gemini(model_name="models/gemini-1.5-flash-latest", api_key=os.environ["GOOGLE_API_KEY"])
gemini.complete("Hi")

CompletionResponse(text='Hi! 👋  What can I do for you today? 😊 \n', additional_kwargs={}, raw={'content': {'parts': [{'text': 'Hi! 👋  What can I do for you today? 😊 \n'}], 'role': 'model'}, 'finish_reason': 1, 'index': 0, 'safety_ratings': [], 'token_count': 0, 'grounding_attributions': [], 'block_reason': 0}, logprobs=None, delta=None)

In [29]:
prompt = f"""
You are a professional researcher in the field of AI. You are given a list of paper abstract in one day.
Your job is to summarize the trends and note out some most interesting papers in the list. 
Give the link to the full paper in the report.
===============
{daily_paper_content}
"""

response = gemini.complete(prompt)

In [30]:
from IPython.display import Markdown
Markdown(response.text)

## AI Research Paper Trends: May 14, 2024

This report summarizes trends and highlights interesting papers from a list of AI research abstracts published on May 14, 2024.

**Trends:**

* **Multimodal Learning:** There is a strong focus on multimodal learning, particularly combining vision and language. This includes papers on vision-language pre-training, scientific figure interpretation, and multimodal large language models for medical applications.
* **Robustness and Generalization:** Many papers address the challenges of robustness and generalization in AI models. This includes work on out-of-distribution detection, domain generalization, and improving the robustness of models to data variations.
* **Explainability and Fairness:**  There is growing interest in making AI models more explainable and fair. Papers explore methods for explaining model decisions, mitigating bias in medical image classification, and ensuring fairness in channel pruning.
* **Reinforcement Learning:**  Reinforcement learning continues to be a popular area of research, with papers focusing on improving efficiency, addressing challenges in offline RL, and applying RL to real-world problems like autonomous intersection management and option hedging.
* **Large Language Models:**  LLMs are being applied to a wide range of tasks, including text-to-image generation, scientific figure interpretation, and enhancing the accessibility of legal documents.

**Most Interesting Papers:**

* **SciFIBench: Benchmarking Large Multimodal Models for Scientific Figure Interpretation** ([http://arxiv.org/abs/2405.08807v1](http://arxiv.org/abs/2405.08807v1)): This paper introduces a new benchmark for evaluating the ability of large multimodal models to understand and interpret scientific figures. This is a crucial area for AI to assist scientific research.
* **CinePile: A Long Video Question Answering Dataset and Benchmark** ([http://arxiv.org/abs/2405.08813v1](http://arxiv.org/abs/2405.08813v1)): This paper presents a new dataset and benchmark for long-form video understanding, which is a challenging task for current AI models. The dataset is designed to test genuine long-form comprehension, not just analyzing a few frames.
* **EfficientTrain++: Generalized Curriculum Learning for Efficient Visual Backbone Training** ([http://arxiv.org/abs/2405.08768v1](http://arxiv.org/abs/2405.08768v1)): This paper proposes a novel approach to curriculum learning that significantly reduces the training time of visual backbones without sacrificing accuracy. This is a valuable contribution to making AI models more efficient.
* **Hunyuan-DiT: A Powerful Multi-Resolution Diffusion Transformer with Fine-Grained Chinese Understanding** ([http://arxiv.org/abs/2405.08748v1](http://arxiv.org/abs/2405.08748v1)): This paper presents a new text-to-image diffusion transformer that demonstrates fine-grained understanding of both English and Chinese. This is a significant step towards developing more multilingual and culturally sensitive AI models.
* **Promoting AI Equity in Science: Generalized Domain Prompt Learning for Accessible VLM Research** ([http://arxiv.org/abs/2405.08668v1](http://arxiv.org/abs/2405.08668v1)): This paper proposes a framework for adapting vision-language models to specialized domains without requiring extensive data or resources. This is a crucial step towards making AI research more accessible and equitable.

**Overall, this day's research highlights the rapid progress being made in AI, particularly in areas like multimodal learning, robustness, explainability, and fairness. These advancements are paving the way for AI to play an increasingly important role in various fields, from scientific research to healthcare and beyond.** 


In [32]:
with open("../outputs/daily_report_may_14.md", "w") as f:
    f.write(response.text)